In [ ]:
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors

from pathlib import Path
import pandas as pd

plt.style.use('ggplot')
pd.set_option('display.max_columns', None)

In [ ]:
INF = 999999

def plot_explored(f, steps=-1, box=INF, l=6, lbox=0, path=True, matches=True, name=""):
    t = pd.read_csv(f, delimiter=',', dtype=None)
    total_t = len(t.index)
 
    max_steps = len(t) if steps == -1 else steps
    
    if box == INF:
        n, m = max(t[:,0]) + 1, max(t[:,1]) + 1
    else:
        n, m = box-lbox, box-lbox
    
    img = np.ones((n, m, 4))
    img.fill(0.97)  # grey
    
    def ok(x): return (lbox <= x) & (x < box)
    
    subt = t.loc[ok(t.i) & ok(t.j)]
    
    min_step = INF
    for index, row in subt.loc[subt['type'] == 'Explored'].iterrows():
        assert row.step != -1
        min_step = min(min_step, row.step)
    
    for index, row in subt.loc[subt['type'] == 'Explored'].iterrows():
        assert row.step != -1
        if row.step > max_steps:
            continue
        val = (row.step-min_step) / (max_steps-min_step)
        if val <= 1.0: 
            img[row.i-lbox, row.j-lbox] = cm.turbo(0.25+(val*1.3)) # cm.ocean(0.0+1.5*val)  # cm.jet(0.2+(val*1.5)) # cm.summer(val**0.5) #cm.cool(val)
        
    if path:
        for index, row in subt.loc[subt['type'] == 'Path'].iterrows():
            img[row.i-lbox, row.j-lbox] = mcolors.to_rgba('xkcd:black')
     
    if matches:
        for index, row in subt.loc[subt['type'] == 'Match'].iterrows():
            for d in range(l):
                if row.i+d-lbox < img.shape[0] and row.j+d-lbox < img.shape[1]:
                    img[row.i+d-lbox, row.j+d-lbox] = mcolors.to_rgba('xkcd:canary')
    
    fig, ax = plt.subplots()
    title = f.stem
    #ax.set_title(name, y=0.02, fontsize=11)
    #ax.xaxis.tick_top()
    ax.grid(False)
    #ax.tick_params(axis='both', which='major', labelsize=11)
    ax.tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, right=False, left=False, labelleft=False)
    #ax.set_xticks([0, 250, 500])
    #ax.set_yticks([0, 250, 500])
    
    ax.imshow(img)
    fig.savefig('imgs/'+title+'.png', bbox_inches = 'tight', dpi=1000)